In [17]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from datetime import datetime
import re
import random
import math

In [18]:
# loading data
codes = pd.read_csv('../results/patients_w_codes.csv')

In [19]:
# version with dates into strings
codes2 = pd.read_csv('../results/patients_w_codes2.csv')

In [20]:
# to use a dict with the dates in the strings
def separate_date(df):
    df['first_event_date'] = 'NA'
    for i in range(len(df)):
        x = re.split(']', df['code'][i].replace('[', ''))
        df['first_event_date'][i] = x[0]
        df['code'][i] = x[1]
    return df

In [21]:
separate_date(codes2)
codes2

,id,code,first_event_date
0,005cad4958846409,B42A40A2H1F10A15B21B7B12I24I7F6B28B128A4D77B64...,2020-08-14
1,0065b57e5a9e784f,A37B234F4B4B8B2F2B2F3B4F0G3B2I8G17I215I82B9B23A,2020-03-26
2,006e5a1bf72e8b73,B56B7B35B11B3B11B10B13B14B8A178I21I123B6B5B6B5...,2020-06-02
3,009d11a2497f1d47,A334C142J,2020-10-14
4,00a5cf0620b6adfc,A201J213A,2020-10-26
...,...,...,...
3298,ff32e199c9482c20,B196B12B21B6F1B7B1I6B6A15B7B7I70B14B78B7B7B8B5...,2020-05-07
3299,ff46357762ea097e,B92B99B12B9B6B4B32F6I4B18I232C4A14B17I79D36B3B...,2020-04-20
3300,ff9318bd975b7050,A16A64B74B14A197I,2020-08-10
3301,ffabda76a8fbaecd,B24B182A12B13B7B6F1B0I7B1B5B7F1I56B15F218B4F10...,2020-05-05


In [22]:
# Function to split strings
def split_string(s):
    """
    s is a string code. Returns two lists, with letters (A) and numbers (B).
    """
    S = []
    for x in re.split('(\d+)', s):
        try:
            S.append(int(x))
        except ValueError:
            S.append(x)
    A, B = S[::2], S[1::2]
    return A, B

In [23]:
def case1(A, B, pos):
    """
    Recognises case 1 reinfections (+|-|+).
    Added some lines to detect certain cases 2.
    """
    if A[0] in pos:
        i = 0
        c = B[i]
        # to find case 2 before case 1
        #if A[i] in pos and A[i+1] in pos and B[i] >= 90:
        #    return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
        if A[i+1] in pos and c >= 90:
            return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
        while A[i+1] != 'B':
            i += 1
            c += B[i]
            # to find case 2 before case 1
            if A[i+1] in pos and c >= 90:
                return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
        # here the first B is found
        i += 1
        c += B[i]
        while A[i+1] not in pos:
            i += 1
            c += B[i]
        if c >= 90:
            return 'Case 1', A[i+1:len(A)], B[i+1:len(B)]
        else:
            return 'Possible case 1', A[i+1:len(A)], B[i+1:len(B)]
    else:
        i = 0
        while A[i] not in pos:
            i += 1
        c = B[i]
        # same as above
        if A[i+1] in pos and c >= 90:
            return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
        while A[i+1] != 'B':
            i += 1
            c += B[i]
            if A[i+1] in pos and c >= 90:
                return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
        i += 1
        c += B[i]
        while A[i+1] not in pos:
            i += 1
            c += B[i]
        if c >= 90:
            return 'Case 1', A[i+1:len(A)], B[i+1:len(B)]
        else:
            return 'Possible case 1', A[i+1:len(A)], B[i+1:len(B)]

In [65]:
def case2(A, B, pos):
    """
    Recognises case 2 reinfections (+|+)
    """
    c = 0
    if A[0] in pos:
        for i in range(len(B)):
            if A[i+1] not in pos:
                if bool([x for x in A[i+1:len(A)] if(x in pos)]) == False:
                    if c >= 60:
                        return 'Possible Case 2', A[i:len(A)], B[i:len(B)]
                    else:
                        return '', A[i:len(A)], B[i:len(B)]
                c += B[i]
            else:
                if c + B[i] >= 90:
                    return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
                c = B[i]
        if c >= 60:
            return 'Possible Case 2', A[i+1:len(A)], B[i+1:len(B)]
        else:
            return '', A[i+1:len(A)], B[i+1:len(B)]
    else:
        i = 0
        while A[i] not in pos:
            i += 1
        for j in range(i, len(B)):
            if A[j+1] not in pos:
                if bool([x for x in A[j+1:len(A)] if(x in pos)]) == False:
                    if c >= 60:
                        return 'Possible Case 2', A[j:len(A)], B[j:len(B)]
                    else:
                        return '', A[j:len(A)], B[j:len(B)]
                c += B[j]
            else:
                if c + B[j] >= 90:
                    return 'Case 2', A[j+1:len(A)], B[j+1:len(B)]
                c = B[j]
        if c >= 60:
            return 'Possible Case 2', A[j+1:len(A)], B[j+1:len(B)]
        else:
            return '', A[j+1:len(A)], B[j+1:len(B)]

In [66]:
# adding multiple reinfection management
def classif(s, pos = ['A', 'C', 'D']):
    """
    Takes a string and classify it into reinfection cases 1 or 2
    """
    A, B = split_string(s)
    L = []
    while len(A) > 1:
        # indices stores a list between the first and last positive in the string
        indices = [i for i, x in enumerate(A) if x in pos]
        # b will be used to break the loop if there are no positives in the remaining string
        b = [x for x in A[1:len(A)] if(x in pos)]
        if 'B' in A[min(indices):max(indices)]:
            r, A, B = case1(A, B, pos)
            L.append(r)
            print('Case 1', A)
        elif bool(b) == False:
            break
        else:
            r, A, B = case2(A, B, pos)
            print('Case 2', A)
            if r != '':
                L.append(r)
    return L

In [49]:
# trying random strings
r = random.randint(0,3303)
print(codes.iloc[r]['id'], '\n')
s = codes.iloc[r]['code']

A, B = split_string(s)

print(s, '\n')

classif(s)

f428362a3b7d8220 

B14B83B42B37A12A2B13A17B14B7B216B113J21B3B7B4B4A3A 

Case 1 ['A', 'B', 'B', 'B', 'B', 'J', 'B', 'B', 'B', 'B', 'A', 'A']
Case 1 ['A', 'A']
Case 2 ['A']


['Possible case 1', 'Case 1']

In [67]:
# applying function
d = {}
for x in codes['id']:
    d[str(x)] = classif(codes.loc[codes['id'] == x]['code'].values[0])

Case 1 ['A', 'D', 'B', 'J', 'B', 'B', 'F', 'F', 'F', 'F', 'F', 'F']
Case 2 ['D', 'B', 'J', 'B', 'B', 'F', 'F', 'F', 'F', 'F', 'F']
Case 1 ['A']
Case 1 ['A', 'J']
Case 2 ['C', 'J']
Case 2 ['A']
Case 2 ['A', 'J', 'F']
Case 2 ['A', 'I', 'I', 'I']
Case 2 ['A', 'G', 'I', 'F']
Case 1 ['A', 'E']
Case 2 ['A', 'B', 'E', 'U']
Case 1 ['A', 'B']
Case 1 ['A', 'B', 'F', 'F', 'F']
Case 2 ['D', 'I', 'I']
Case 1 ['A', 'B', 'J']
Case 2 ['D', 'J']
Case 1 ['A', 'H', 'I', 'J', 'F', 'F']
Case 1 ['A', 'F']
Case 2 ['C', 'F']
Case 1 ['A', 'E']
Case 1 ['A']
Case 2 ['A']
Case 1 ['C', 'B', 'B', 'B', 'F', 'F', 'B', 'B', 'T']
Case 2 ['A', 'I', 'F', 'U', 'I']
Case 2 ['A']
Case 2 ['A', 'A', 'I', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']
Case 2 ['A', 'I', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']
Case 1 ['A', 'I', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']
Case 2 ['A', 'F']
Case 1 ['A']
Case 1 ['C']
Case 1 ['D']
Case 1 ['A']
Case 1 ['A', 'C', 'A', 'A', 'B', 'B']
Case 2 ['A', 'B', 'B']
Case 2 ['A', 'E']
Case 2 

Case 1 ['A']
Case 2 ['A', 'I']
Case 1 ['A']
Case 1 ['A', 'B', 'H', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'I', 'F', 'B', 'F', 'B', 'B', 'F', 'I', 'B', 'G', 'B', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'B', 'B', 'B', 'B', 'I', 'B', 'B', 'F', 'F', 'B', 'B']
Case 1 ['C']
Case 2 ['A', 'B']
Case 1 ['A', 'B', 'B', 'B', 'B', 'I', 'A', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'F', 'F', 'F', 'F', 'F', 'F', 'I', 'F']
Case 1 ['A', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'F', 'F', 'F', 'F', 'F', 'F', 'I', 'F']
Case 1 ['A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'F', 'F', 'F', 'F', 'F', 'F', 'I', 'F']
Case 1 ['A', 'B', 'A', 'J', 'B']
Case 1 ['A', 'J', 'B']
Case 1 ['C', 'N', 'Q', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B']
Case 2 ['A', 'B', 'B', 'B', 'B', 'B']
Case 1 ['C']
Case 2 ['A', 'H', 'I', 'J']
Case 1 ['A']
Case 2 ['A']
Case 1 ['C']
Case 1 ['A'

Case 1 ['C', 'U', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'J']
Case 1 ['A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'J']
Case 1 ['A', 'J']
Case 1 ['D']
Case 2 ['A', 'G', 'I']
Case 1 ['A', 'B']
Case 1 ['A']
Case 1 ['A', 'G', 'F', 'B', 'F', 'I', 'B', 'B', 'F', 'F', 'F', 'F', 'F', 'F', 'I', 'B', 'B', 'B', 'A']
Case 1 ['A']
Case 1 ['C', 'J']
Case 1 ['A']
Case 1 ['A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'F', 'B', 'F', 'F', 'B', 'F', 'F', 'B', 'J', 'F', 'A', 'F', 'F']
Case 1 ['A', 'F', 'F']
Case 1 ['A']
Case 1 ['A', 'B']
Case 1 ['D', 'J', 'B', 'B', 'B', 'B', 'B']
Case 1 ['A', 'E', 'B', 'G', 'B', 'I', 'B', 'B', 'F', 'I', 'B', 'B', 'B', 'B', 'B', 'I', 'B', 'B', 'B']
Case 2 ['A', 'F']
Case 1 ['A', 'P', 'Q']
Case 1 ['A', 'F']
Case 2 ['A', 'A']
Case 2 ['A']
Case 1 ['A']
Case 1 ['A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'J', 'F', 'B', 'B', 'A', 'A', 'A', 'B', 'B', 'F', 'F', 'F']
Case 1 ['A', 'A', 'A', 'B', 'B', 'F', 'F', 'F']
Case 2 ['A', 'B', 'B

Case 2 ['A', 'I', 'B']
Case 2 ['D']
Case 2 ['A']
Case 2 ['A', 'I', 'J']
Case 1 ['A']
Case 1 ['A']
Case 2 ['C']
Case 1 ['A', 'H', 'B', 'L']
Case 1 ['A']
Case 1 ['A', 'B', 'B', 'B', 'F', 'J', 'F', 'A']
Case 1 ['A']
Case 1 ['A', 'E', 'F', 'B', 'B']
Case 2 ['A', 'I', 'F']
Case 1 ['A', 'B', 'I', 'B', 'F', 'B', 'B', 'F', 'F', 'B', 'B', 'B', 'B', 'B', 'B']
Case 2 ['A']
Case 1 ['A', 'G', 'B', 'B', 'Q']
Case 2 ['A', 'A', 'H']
Case 2 ['A', 'H']
Case 1 ['A']
Case 1 ['A', 'F']
Case 1 ['A', 'I', 'I', 'A', 'B', 'I', 'B', 'B', 'B']
Case 2 ['A', 'B', 'I', 'B', 'B', 'B']
Case 1 ['A', 'I', 'G', 'I', 'A', 'B', 'G', 'F', 'F', 'J', 'E', 'E']
Case 2 ['A', 'B', 'G', 'F', 'F', 'J', 'E', 'E']
Case 1 ['A']
Case 2 ['A', 'G', 'F', 'B', 'F', 'J', 'F', 'E', 'F']
Case 1 ['A', 'B', 'G', 'B', 'B', 'B', 'J']
Case 1 ['C', 'A', 'A', 'B', 'J', 'B', 'B', 'B']
Case 2 ['A', 'B', 'J', 'B', 'B', 'B']
Case 1 ['C', 'F', 'F']
Case 1 ['A', 'B', 'B', 'B', 'B', 'B', 'B']
Case 1 ['A', 'I', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',

Case 1 ['A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'I', 'A', 'E']
Case 1 ['A', 'E']
Case 1 ['D']
Case 1 ['A']
Case 1 ['A', 'B', 'I', 'B', 'B', 'B', 'B', 'B']
Case 2 ['A', 'G', 'G', 'K', 'F', 'J']
Case 1 ['A', 'B']
Case 2 ['A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'J']
Case 1 ['A', 'F', 'I', 'I', 'B', 'B', 'B', 'B', 'T']
Case 1 ['A']
Case 1 ['A']
Case 2 ['A']
Case 1 ['A', 'A', 'A', 'G', 'B', 'B', 'G', 'A', 'B', 'G', 'I', 'A', 'G', 'I', 'G', 'B', 'J', 'B', 'B', 'B', 'B', 'B', 'B', 'A']
Case 1 ['A', 'B', 'G', 'I', 'A', 'G', 'I', 'G', 'B', 'J', 'B', 'B', 'B', 'B', 'B', 'B', 'A']
Case 1 ['A', 'G', 'I', 'G', 'B', 'J', 'B', 'B', 'B', 'B', 'B', 'B', 'A']
Case 1 ['A']
Case 1 ['A', 'A', 'A', 'B']
Case 2 ['A', 'B']
Case 1 ['A', 'G', 'B', 'B', 'B', 'B', 'F', 'B', 'B', 'F', 'F', 'B', 'I', 'F', 'B', 'F', 'B', 'F', 'B', 'I', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'F', 'A', 'F', 'E', 'B']
Case 1 ['A', 'F', 'E', 'B']


...

Case 1 ['A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'J', 'B', 'E', 'A', 'E', 'B', 'A', 'B']
Case 1 ['A', 'E', 'B', 'A', 'B']
Case 1 ['A', 'B']
Case 1 ['A']
Case 1 ['A']
Case 1 ['A']
Case 1 ['A']
Case 1 ['A', 'B', 'I', 'B', 'F', 'B', 'B', 'I', 'F', 'F', 'B', 'A', 'B', 'B', 'F', 'F', 'F', 'J', 'F', 'B', 'B', 'F', 'B', 'F', 'F']
Case 1 ['A', 'B', 'B', 'F', 'F', 'F', 'J', 'F', 'B', 'B', 'F', 'B', 'F', 'F']
Case 1 ['A', 'C']
Case 2 ['C']
Case 1 ['A']
Case 1 ['A', 'A', 'B', 'F', 'F', 'F', 'F']
Case 2 ['A', 'B', 'F', 'F', 'F', 'F']
Case 2 ['A', 'J', 'B']
Case 1 ['A', 'F', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'J', 'B', 'F', 'F', 'B', 'A', 'F', 'F']
Case 1 ['A', 'F', 'F']
Case 1 ['A', 'A', 'B', 'F', 'B', 'I', 'B', 'B', 'B', 'I', 'B', 'B', 'I', 'F', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'F', 'B', 'B', 'A']
Case 1 ['A']
Case 1 ['A', 'A', 'A', 'B']
Case 2 ['A', 'B']
Case 1 ['A']
Case 1 ['A', 'A']
Case 2 ['A']
Case 1 ['A', 'B']
Case 1 ['A']
Ca

Case 2 ['A']
Case 1 ['A', 'J']
Case 1 ['A', 'G', 'F', 'F', 'F', 'F', 'K', 'K', 'J']
Case 2 ['A', 'B', 'I']
Case 1 ['A']
Case 1 ['C', 'F']
Case 2 ['A', 'F', 'B', 'G', 'I', 'B', 'F', 'I']
Case 1 ['D', 'A', 'B']
Case 2 ['A', 'B']
Case 2 ['A', 'G']
Case 1 ['A', 'E']
Case 1 ['C']
Case 1 ['A']
Case 1 ['A']
Case 1 ['A', 'B', 'G', 'B', 'I', 'B', 'B', 'B', 'B', 'I', 'F', 'F', 'B', 'E']
Case 2 ['A']
Case 1 ['A']
Case 1 ['C', 'F']
Case 1 ['A']
Case 1 ['A']
Case 1 ['A']
Case 2 ['A']
Case 1 ['A', 'E', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'F', 'B', 'B', 'B', 'B', 'B', 'B']
Case 2 ['A', 'B', 'B', 'B', 'B', 'B', 'B', 'F', 'B', 'B', 'B', 'B', 'B', 'B']
Case 2 ['A']
Case 1 ['A', 'Q']
Case 1 ['A', 'B', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'A']
Case 1 ['A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'A']
Case 1 ['A', 'B', 'A', 'B', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'A']
Case 1 ['A', 'B', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'A']
Case 1 ['A

Case 1 ['C']
Case 1 ['A']
Case 2 ['A']
Case 1 ['A']
Case 1 ['A', 'B']
Case 1 ['A', 'I']
Case 1 ['A', 'B', 'B', 'B', 'B']
Case 1 ['A']
Case 1 ['A']
Case 1 ['A']
Case 1 ['C']
Case 2 ['A']
Case 1 ['A', 'G', 'F', 'B', 'B', 'F', 'F', 'B', 'B', 'F', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'J', 'B', 'B', 'B', 'B', 'B', 'B']
Case 1 ['D', 'B', 'B', 'B', 'B', 'B', 'B', 'F', 'F', 'F', 'F']
Case 1 ['A', 'A']
Case 2 ['A']
Case 1 ['A', 'G', 'I', 'I', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'J', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']
Case 1 ['A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'J', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']
Case 2 ['A', 'F', 'F']
Case 1 ['A']
Case 1 ['A']
Case 2 ['A', 'B', 'F', 'B', 'J']
Case 1 ['A']
Case 2 ['C', 'I', 'I', 'F', 'F']
Case 2 ['A', 'I', 'I', 'F', 'J', 'B']
Case 1 ['A', 'I', 'B', 'H', 'B', 'I', 'B', 'B', 'I', 'F', 'B', 'B', 'B', 'B', 'B', 'B

Case 2 ['A']
Case 1 ['A', 'F']
Case 1 ['C']
Case 1 ['A', 'B', 'B', 'I', 'I', 'B', 'B', 'J', 'B', 'B', 'B', 'B']
Case 1 ['C', 'A', 'A', 'F', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'F', 'F', 'B', 'E', 'F', 'F', 'F']
Case 2 ['A', 'F', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'F', 'F', 'B', 'E', 'F', 'F', 'F']
Case 1 ['A', 'B', 'H', 'I', 'I', 'B']
Case 1 ['A', 'B', 'B', 'B', 'B', 'J', 'B', 'B', 'B', 'A', 'F']
Case 1 ['A', 'F']
Case 2 ['C', 'I', 'J']
Case 1 ['A']
Case 1 ['A']
Case 1 ['A', 'F', 'F', 'F']
Case 2 ['A', 'J']
Case 1 ['A', 'B', 'B']
Case 1 ['C', 'J', 'J', 'B', 'T']
Case 2 ['C', 'C', 'I', 'F', 'F', 'I', 'F']
Case 2 ['C', 'I', 'F', 'F', 'I', 'F']
Case 1 ['C', 'E']
Case 1 ['D']
Case 1 ['A', 'T']
Case 2 ['A', 'F']
Case 1 ['A', 'F', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'J', 'B', 'B', 'B', 'B', 'F', 'B', 'B', 'F', 'F', 'F', 'B', 'F', 'F', 'F', 'F', 'F', 'F', 'F']
Case 1 ['C', 'B', 'G', 'J', 'B', 'B', 'B', 'F', 'T'

Case 2 ['A', 'E', 'A', 'A', 'A', 'A']
Case 2 ['A']
Case 1 ['A', 'F']
Case 1 ['A']
Case 2 ['A', 'B', 'G', 'B', 'B', 'E']
Case 1 ['A', 'B']
Case 1 ['A']
Case 1 ['A', 'A', 'F']
Case 2 ['A', 'F']
Case 1 ['A', 'B', 'J']
Case 1 ['A', 'C', 'H', 'I', 'F', 'E', 'F']
Case 2 ['C', 'H', 'I', 'F', 'E', 'F']
Case 2 ['A']
Case 2 ['C', 'B', 'B', 'G', 'B', 'F', 'B']
Case 1 ['A', 'B']
Case 1 ['A', 'B']
Case 1 ['A', 'A', 'A', 'B', 'B', 'A', 'A']
Case 1 ['A', 'A']
Case 2 ['A']
Case 2 ['A']
Case 1 ['A', 'J']
Case 2 ['A']
Case 1 ['A']
Case 1 ['A']
Case 2 ['A', 'B', 'J', 'F', 'F', 'F', 'F']
Case 1 ['D', 'E', 'F']
Case 2 ['D', 'F']
Case 1 ['C']
Case 2 ['A', 'J']
Case 2 ['A', 'B', 'I', 'J']
Case 1 ['A', 'B', 'B', 'B', 'B', 'J', 'B', 'B', 'B', 'A', 'B', 'B', 'B']
Case 1 ['A', 'B', 'B', 'B']
Case 1 ['A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'J', 'B', 'B', 'B', 'B', 'B', 'F', 'F', 'B', 'B']
Case 1 ['A', 'F', 'B', 'F', 'B', 'B', 'B', 'B', 'B']
Case 2 ['C']
Case 1 ['A', 'F', 'B']
Case 1 ['A']
Case

In [28]:
import csv

with open('../results/dict.csv', 'w') as f:
    for key in d.keys():
        f.write("%s, %s\n" % (key, d[key]))

In [68]:
N = list(d.values())
N = [str(x) for x in N]
freq = {}
for items in N:
    freq[items] = N.count(items)
freq

{"['Case 1']": 2014,
 "['Case 2']": 987,
 "['Case 2', 'Case 2']": 5,
 "['Case 1', 'Case 1']": 86,
 "['Possible case 1', 'Case 2']": 6,
 "['Case 1', 'Possible case 1']": 41,
 "['Possible case 1', 'Case 1']": 59,
 "['Possible case 1', 'Possible case 1']": 13,
 "['Case 2', 'Case 1']": 16,
 "['Case 1', 'Possible case 1', 'Case 1']": 5,
 "['Case 2', 'Possible case 1']": 3,
 "['Possible case 1', 'Case 1', 'Possible case 1']": 3,
 "['Possible case 1', 'Possible case 1', 'Possible case 1']": 3,
 "['Possible case 1']": 13,
 '[]': 14,
 "['Possible case 1', 'Case 1', 'Case 1']": 5,
 "['Case 1', 'Case 2']": 2,
 "['Case 1', 'Case 1', 'Possible case 1', 'Possible case 1']": 1,
 "['Case 2', 'Possible case 1', 'Case 1']": 1,
 "['Possible Case 2']": 6,
 "['Possible case 1', 'Possible case 1', 'Case 1']": 7,
 "['Case 1', 'Case 1', 'Possible case 1']": 4,
 "['Case 1', 'Possible case 1', 'Possible case 1', 'Case 1']": 1,
 "['Case 2', 'Possible Case 2']": 1,
 "['Possible case 1', 'Possible case 1', 'Possib